In [1]:
import pandas as pd
import numpy as np
import os

### Cache Series Data

In [3]:
dirList = os.listdir("data/Transformed/")
fList = []
for filename in dirList: #limit to .csv
    if(filename[-4:] == '.csv'):
        fList.append(filename[:-4]) 

In [5]:
dfs = []
count = 0
for fName in fList[:]:
    fContent = pd.read_csv('data/Transformed/'+fName+'.csv')
#     fContent.columns = ['ContestId', 'SecondsRemaining', 'Entries']
    count = count+len(fContent)
    dfs.append(fContent)
seriesData = pd.concat(dfs).set_index('ContestId')


In [6]:
seriesData.head()

,0HoursOut,10HoursOut,12HoursOut,16HoursOut,1HoursOut,20HoursOut,24HoursOut,2HoursOut,36HoursOut,3HoursOut,48HoursOut,4HoursOut,5HoursOut,6HoursOut,7HoursOut,8HoursOut,9HoursOut
ContestId,,,,,,,,,,,,,,,,,
55826033,46576,3713,2427,814,17049,0,0,11604,0,8863,0,7377,6808,5799,5298,4871,4141
56610037,2607,0,0,0,1717,0,0,1100,0,777,0,624,515,429,291,157,0
56645954,72,0,0,0,65,0,0,55,0,35,0,0,0,0,0,0,0
56751298,103,0,0,0,17,0,0,11,0,7,0,2,1,0,0,0,0
56776765,2497,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
contests = pd.read_csv('data/WorkingData.csv').set_index('ContestId')[['TotalPrizeAmount', 'EntryFeeAmount']]
contests = contests.assign(SuccessThreshold= lambda x:(contests['TotalPrizeAmount'].astype(float)/(contests['EntryFeeAmount'].astype(float))).astype(int))

In [8]:
combinedDF = pd.merge(contests[['SuccessThreshold']], seriesData, on='ContestId', how='right').dropna()

In [9]:
combinedDF.head()

,SuccessThreshold,0HoursOut,10HoursOut,12HoursOut,16HoursOut,1HoursOut,20HoursOut,24HoursOut,2HoursOut,36HoursOut,3HoursOut,48HoursOut,4HoursOut,5HoursOut,6HoursOut,7HoursOut,8HoursOut,9HoursOut
ContestId,,,,,,,,,,,,,,,,,,
7963004,100000.0,111509,24065,17185,3900,83405,0,0,67175,0,57766,0,52020,44665,40591,35741,31261,27972
7963437,12500.0,8677,1872,1295,289,5993,0,0,4810,0,4234,0,3735,3243,3034,2729,2489,2136
8296224,50.0,53,7,0,0,44,0,0,35,0,30,0,27,22,22,22,18,11
8296544,100.0,25,1,0,0,17,0,0,12,0,10,0,8,6,6,3,3,1
8494213,60000.0,19067,0,0,0,9277,0,0,5991,0,3851,0,2946,1911,1396,788,67,0


In [10]:
scaleFeatures = combinedDF.columns.drop('SuccessThreshold')
for f in scaleFeatures:
    combinedDF = combinedDF.assign(newCol= lambda x:(combinedDF[f].astype(float)/(combinedDF['SuccessThreshold'].astype(float))).astype(float))
    combinedDF = combinedDF.rename(columns={'newCol': f+'_scaled'}).drop(columns=[f])
                                            

In [11]:
hoursOutDf = combinedDF.drop(columns='SuccessThreshold')

In [12]:
workedDF = pd.read_csv('data/WorkedData.csv').set_index('ContestId')


In [13]:
combinedDF = pd.merge(workedDF, hoursOutDf, on='ContestId', how='right').dropna()
combinedDF.head()

,Success,SportName_NFL,SportName_PGA,SportName_SOCC,SportName_MLB,SportName_NAS,SportName_CFB,SportName_MMA,SportName_LOL,SportName_NHL,...,2HoursOut_scaled,36HoursOut_scaled,3HoursOut_scaled,48HoursOut_scaled,4HoursOut_scaled,5HoursOut_scaled,6HoursOut_scaled,7HoursOut_scaled,8HoursOut_scaled,9HoursOut_scaled
ContestId,,,,,,,,,,,,,,,,,,,,,
7963004,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.67175,0.0,0.577660,0.0,0.5202,0.44665,0.405910,0.357410,0.312610,0.27972
7963437,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.38480,0.0,0.338720,0.0,0.2988,0.25944,0.242720,0.218320,0.199120,0.17088
8296224,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.70000,0.0,0.600000,0.0,0.5400,0.44000,0.440000,0.440000,0.360000,0.22000
8296544,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.12000,0.0,0.100000,0.0,0.0800,0.06000,0.060000,0.030000,0.030000,0.01000
8494213,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.09985,0.0,0.064183,0.0,0.0491,0.03185,0.023267,0.013133,0.001117,0.00000


In [14]:
combinedDF.to_csv('data/SCALEDEVERYTHING_SECONDS.csv')